# Research Project 



#### Finding data
- BTC price from API? or from different sources (i started on this - trying to get daily prices from api)
- Oil, Gold, S&P 500 from Yahoo/Crsp/FactSet?
- Google trend - we can find google trend also through api --> pytrends: https://pypi.org/project/pytrends/
- Inflation, Interest rate in the US - CRSP or some other webpage with reliable info
+ Deciding whether we will use returns/ prices/ so on :)

In [1]:
#pip install pandas-datareader

In [2]:
import pandas_datareader.data as web
import pandas as pd
import datetime as dt
import os
import numpy as np

In [3]:
# yahoo finance data
df1 = web.DataReader(['GC=F', 'CL=F', '^GSPC'], 'yahoo', start='2019-12-31', end='2022-01-18')["Adj Close"]
df1.rename(columns={'GC=F': 'GOLD', 'CL=F': 'OIL', '^GSPC': 'SP500'}, inplace=True)

In [4]:
# df1.tail()
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 518 entries, 2019-12-31 to 2022-01-18
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GOLD    518 non-null    float64
 1   OIL     518 non-null    float64
 2   SP500   516 non-null    float64
dtypes: float64(3)
memory usage: 16.2 KB


In [5]:
returns = df1.copy()
for i in range(1,len(df1+1)):
    returns.iloc[i] = np.log(df1.iloc[i]/df1.iloc[i-1])
returns = returns[1:]

C:\Users\mezyn\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
# effective federal funds rate (https://fred.stlouisfed.org/series/EFFR)
df2 = web.DataReader(['EFFR'], 'fred', start='2020-01-01', end='2022-01-01')
df2.rename(columns={'EFFR': 'IR'}, inplace=True)
df2['IR']= df2['IR']/100

In [7]:
df2.head()

,IR
DATE,
2020-01-01,NaN
2020-01-02,0.0155
2020-01-03,0.0155
2020-01-06,0.0155
2020-01-07,0.0155


In [8]:
df = returns.merge(df2, how= 'left', left_index = True, right_index = True)

In [9]:
# df.info()
df.head()
# df.tail()

,GOLD,OIL,SP500,IR
Date,,,,
2020-01-02,0.003285,0.001963,0.008344,0.0155
2020-01-03,0.016072,0.030108,-0.007085,0.0155
2020-01-06,0.010914,0.003483,0.003527,0.0155
2020-01-07,0.003569,-0.009050,-0.002807,0.0155
2020-01-08,-0.009204,-0.050538,0.004890,0.0155


In [22]:
df3 = web.DataReader(['CPIAUCSL'], 'fred', start='2019-11-30', end='2022-01-01')
df3.rename(columns={'CPIAUCSL': 'INFL'}, inplace=True)

In [24]:
infl = df3.copy()
for i in range(1,len(df3+1)):
    infl.iloc[i] = np.log(df3.iloc[i]/df3.iloc[i-1])
infl = infl[1:]
infl.head()

,INFL
DATE,
2020-01-01,0.001873
2020-02-01,0.000529
2020-03-01,-0.003231
2020-04-01,-0.006990
2020-05-01,-0.000976


In [10]:
#pip install pytrends

In [11]:
from pytrends.request import TrendReq
from pytrends import dailydata

Bitcoin_df = dailydata.get_daily_data('Bitcoin', 2020, 1, 2022, 1, geo = '')

Bitcoin:2020-01-01 2020-01-31
Bitcoin:2020-02-01 2020-02-29
Bitcoin:2020-03-01 2020-03-31
Bitcoin:2020-04-01 2020-04-30
Bitcoin:2020-05-01 2020-05-31
Bitcoin:2020-06-01 2020-06-30
Bitcoin:2020-07-01 2020-07-31
Bitcoin:2020-08-01 2020-08-31
Bitcoin:2020-09-01 2020-09-30
Bitcoin:2020-10-01 2020-10-31
Bitcoin:2020-11-01 2020-11-30
Bitcoin:2020-12-01 2020-12-31
Bitcoin:2021-01-01 2021-01-31
Bitcoin:2021-02-01 2021-02-28
Bitcoin:2021-03-01 2021-03-31
Bitcoin:2021-04-01 2021-04-30
Bitcoin:2021-05-01 2021-05-31
Bitcoin:2021-06-01 2021-06-30
Bitcoin:2021-07-01 2021-07-31
Bitcoin:2021-08-01 2021-08-31
Bitcoin:2021-09-01 2021-09-30
Bitcoin:2021-10-01 2021-10-31
Bitcoin:2021-11-01 2021-11-30
Bitcoin:2021-12-01 2021-12-31
Bitcoin:2022-01-01 2022-01-31


In [12]:
Bitcoin_df

,Bitcoin_unscaled,Bitcoin_monthly,isPartial,scale,Bitcoin
date,,,,,
2020-01-01,57,NaN,NaN,NaN,NaN
2020-01-02,67,NaN,NaN,NaN,NaN
2020-01-03,73,NaN,NaN,NaN,NaN
2020-01-04,65,NaN,NaN,NaN,NaN
2020-01-05,67,14.0,False,0.14,9.38
...,...,...,...,...,...
2022-01-11,77,35.0,NaN,0.35,26.95
2022-01-12,71,35.0,NaN,0.35,24.85
2022-01-13,70,35.0,NaN,0.35,24.50


In [15]:
path = os.path.join('data','BTC-USD-2.csv')

In [16]:
df_data = pd.read_csv(path)
df_data.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [17]:
df_data.drop(columns=['Open', 'High', 'Low', 'Adj Close'], inplace=True)

In [18]:
df_data.set_index('Date')

,Close,Volume
Date,,
2020-01-01,7200.174316,18565664997
2020-01-02,6985.470215,20802083465
2020-01-03,7344.884277,28111481032
2020-01-04,7410.656738,18444271275
2020-01-05,7411.317383,19725074095
...,...,...
2022-01-11,42735.855469,26327648900
2022-01-12,43949.101563,33499938689
2022-01-13,42591.570313,47691135082


#### Descriptives - get to know the data

#### Dickey-Fuller test (if needed)

#### Correlation matirx

#### Classic OLS model

#### VAR model

#### If needed also ARDL